## Minería de datos
**Universidad Anáhuac Puebla**

**Dr. Farid Krayem Pineda**

Se hacen la instalación de las librerías necesarias

In [22]:
!pip install pymysql
from IPython.display import clear_output
clear_output()

print("Completada la instalación")

Completada la instalación


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import userdata
from openai import OpenAI
import pymysql

In [24]:
# Configuración de la API key y contraseña de la base de datos
openai_key = userdata.get('api_key_farid')
pass_db = userdata.get('pass_db_anahua')

In [3]:
# Scrapping para recuperar el contenido del calendario de community manager

url = 'https://metricool.com/es/calendario-community-manager-mexico/'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')

data = []
for heading in soup.find_all('h3'):
    mes = heading.get_text(strip=True)
    fig = heading.find_next_sibling('figure', class_='wp-block-table')
    if not fig:
        continue
    table = fig.find('table')
    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        dia = tds[0].get_text(strip=True)
        eventos = tds[1].get_text(separator='|').split('|')
        for evento in eventos:
            evento = evento.strip()
            if evento:
                data.append({'Mes': mes, 'Día': dia, 'Efeméride': evento})

df = pd.DataFrame(data)

In [4]:
df.head()

,Mes,Día,Efeméride
0,Enero 2025,01 de enero,Día de Año Nuevo
1,Enero 2025,04 de enero,Día Mundial del Braille
2,Enero 2025,06 de enero,Día de Reyes
3,Enero 2025,13 de enero,Día Mundial de la Lucha Contra la Depresión
4,Enero 2025,13 de enero,Día del Sticker


In [6]:
# Crear mayo_df con los primeros 10 días de mayo
mayo_df = df[df['Mes'].str.contains('Mayo', case=False)]
mayo_df = mayo_df[mayo_df['Día'].str[:2].astype(int) <= 10]
mayo_df.reset_index(drop=True, inplace=True)
mayo_df

,Mes,Día,Efeméride
0,Mayo 2025,01 de mayo,Día del Trabajo
1,Mayo 2025,02 de mayo,Día Internacional Contra el Acoso Escolar
2,Mayo 2025,03 de mayo,Día de la Libertad de Prensa
3,Mayo 2025,04 de mayo,Día de Star Wars
4,Mayo 2025,05 de mayo,Aniversario de LinkedIn
5,Mayo 2025,05 de mayo,Día Mundial de la Risa
6,Mayo 2025,08 de mayo,Día Mundial de la Cruz Roja y de la Media Luna...
7,Mayo 2025,10 de mayo,Día de la Madre
8,Mayo 2025,10 de mayo,Día Mundial de las Aves Migratorias


In [12]:
# Se crea el cliente necesario para hacer peticiones a la API de OpenAI
client = OpenAI(api_key=openai_key)

In [15]:
# Se crea la función para crear contenidos

def respuesta(nom_brand, desc_brand, efemeride):

  rol = "Tu rol es ser un experto en la creación de contenido de redes sociales."
  prompt = f'''
  Recibes una efeméride, tu tarea es crear un contenido para esa efeméride y darle el contexto según el nombre de mi marca y a lo que me dedico.

  Nombre de la marca = {nom_brand}
  Descripción de la marca = {desc_brand}
  Efeméride = {efemeride}

  La salida es un contenido de 100 palabras en español y con emojis. Hazlo atractivo, el objetivo es alcanzar altos niveles de engagement.

  '''

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
            "role": "user",
            "content": rol + " " + prompt
        }
    ]
  )

  answer = completion.choices[0].message.content

  return answer


In [17]:
# Probando la función con un solo contenido

nom_brand = "Universidad Anáhuac Puebla"
desc_brand = "Universidad con altos niveles de innovación tecnológica. Líderes de acción positiva"
efemeride = f'''
Fecha: {mayo_df.iloc[0]['Día']} de 2025.
Efeméride: {mayo_df.iloc[0]['Efeméride']}'''

contenido = respuesta(nom_brand, desc_brand, efemeride)

print(contenido)

🎉 ¡Feliz Día del Trabajo! En la Universidad Anáhuac Puebla celebramos a todos los trabajadores que día a día dan lo mejor de sí. 💪🏼 En nuestra institución, fomentamos la innovación tecnológica y el liderazgo para formar a los líderes del mañana. 🚀📚 En este día especial, reconocemos la importancia del trabajo en equipo, la dedicación y el esfuerzo para alcanzar nuestras metas. ¡Gracias a todos los que hacen posible que nuestra universidad sea un lugar de acción positiva! 👏🏼 ¡Sigamos trabajando juntos para lograr grandes cosas! 💼 #DíaDelTrabajo #UniversidadAnáhuacPuebla #InnovaciónTecnológica


## Trabajando con el dataframe

In [18]:
# Se crea el for para completar el dataframe de MAYO

nom_brand = "Universidad Anáhuac Puebla"
desc_brand = "Universidad con altos niveles de innovación tecnológica. Líderes de acción positiva"

contenidos = []
for _, row in mayo_df.iterrows():
    efemeride = f'''
Fecha: {row['Día']} de 2025.
Efeméride: {row['Efeméride']}'''
    contenidos.append(respuesta(nom_brand, desc_brand, efemeride))

mayo_df['Contenido'] = contenidos


In [19]:
mayo_df

,Mes,Día,Efeméride,Contenido
0,Mayo 2025,01 de mayo,Día del Trabajo,"🎉 Hoy celebramos el Día del Trabajo, una fecha..."
1,Mayo 2025,02 de mayo,Día Internacional Contra el Acoso Escolar,📚🌟 Hoy conmemoramos el Día Internacional Contr...
2,Mayo 2025,03 de mayo,Día de la Libertad de Prensa,🌟 Hoy celebramos el Día de la Libertad de Pren...
3,Mayo 2025,04 de mayo,Día de Star Wars,🌟 ¡Hoy es el Día de Star Wars! 🌌 En la Univers...
4,Mayo 2025,05 de mayo,Aniversario de LinkedIn,"🎉 Hoy celebramos el Aniversario de LinkedIn, l..."
5,Mayo 2025,05 de mayo,Día Mundial de la Risa,🎉 Hoy celebramos el Día Mundial de la Risa! En...
6,Mayo 2025,08 de mayo,Día Mundial de la Cruz Roja y de la Media Luna...,🌟 Hoy celebramos el Día Mundial de la Cruz Roj...
7,Mayo 2025,10 de mayo,Día de la Madre,🌸 ¡Feliz Día de la Madre! 🌸 En la Universidad ...
8,Mayo 2025,10 de mayo,Día Mundial de las Aves Migratorias,🕊️🌎 Hoy celebramos el Día Mundial de las Aves ...


In [20]:
# Guardar mayo_df como CSV con encoding UTF-8
mayo_df.to_csv('mayo_df.csv', encoding='utf-8', index=False)

# Descargar el archivo en Colab
from google.colab import files
files.download('mayo_df.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Trabajando con SQL

In [28]:
## Probamos la conexión

host = "35.215.110.77"
port = 3306
user = "ugu1kvcishvbx"
password = pass_db
db = "dboewuclpuqgcr"

try:
    conn = pymysql.connect(host=host, port=port, user=user, password=password, database=db)
    cursor = conn.cursor()
    print("Conexión exitosa")
except Exception as e:
    print(f"Error al conectar: {e}")


Conexión exitosa


In [29]:
# Crear tabla si no existe
cursor.execute("""
CREATE TABLE IF NOT EXISTS social_media_anahuac (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Mes VARCHAR(50),
    Dia VARCHAR(20),
    Efemeride TEXT,
    Nom_brand VARCHAR(255),
    Desc_brand TEXT,
    Contenido TEXT
)
""")

# Insertar datos de mayo_df
for _, row in mayo_df.iterrows():
    cursor.execute("""
        INSERT INTO social_media_anahuac
        (Mes, Dia, Efemeride, Nom_brand, Desc_brand, Contenido)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['Mes'],
        row['Día'],
        row['Efeméride'],
        nom_brand,
        desc_brand,
        row['Contenido']
    ))

conn.commit()


In [30]:
pd.read_sql("SELECT * FROM social_media_anahuac", conn)

<ipython-input-30-f2c7d5960851>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM social_media_anahuac", conn)


,id,Mes,Dia,Efemeride,Nom_brand,Desc_brand,Contenido
0,1,Mayo 2025,01 de mayo,Día del Trabajo,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,"🎉 Hoy celebramos el Día del Trabajo, una fecha..."
1,2,Mayo 2025,02 de mayo,Día Internacional Contra el Acoso Escolar,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,📚🌟 Hoy conmemoramos el Día Internacional Contr...
2,3,Mayo 2025,03 de mayo,Día de la Libertad de Prensa,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,🌟 Hoy celebramos el Día de la Libertad de Pren...
3,4,Mayo 2025,04 de mayo,Día de Star Wars,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,🌟 ¡Hoy es el Día de Star Wars! 🌌 En la Univers...
4,5,Mayo 2025,05 de mayo,Aniversario de LinkedIn,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,"🎉 Hoy celebramos el Aniversario de LinkedIn, l..."
5,6,Mayo 2025,05 de mayo,Día Mundial de la Risa,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,🎉 Hoy celebramos el Día Mundial de la Risa! En...
6,7,Mayo 2025,08 de mayo,Día Mundial de la Cruz Roja y de la Media Luna...,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,🌟 Hoy celebramos el Día Mundial de la Cruz Roj...
7,8,Mayo 2025,10 de mayo,Día de la Madre,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,🌸 ¡Feliz Día de la Madre! 🌸 En la Universidad ...
8,9,Mayo 2025,10 de mayo,Día Mundial de las Aves Migratorias,Universidad Anáhuac Puebla,Universidad con altos niveles de innovación te...,🕊️🌎 Hoy celebramos el Día Mundial de las Aves ...


## Resultado

Conectados con el front puedes compartir esto con tu equipo de redes:

https://mercabits.com/dev/social-try/